In [22]:
import os
import sys
import backtrader as bt
import datetime
import mplfinance
import pandas as pd 

In [35]:
class DonchianChannel(bt.Indicator):
    lines = ('upper', 'lower')

    params = (('period', 20),)

    def __init__(self):
        self.addminperiod(self.params.period + 1)  # Ensure enough historical data

    def next(self):
        if len(self) >= self.params.period:
            highest_high = max(self.datas[0].high.get(size=self.params.period))
            lowest_low = min(self.datas[0].low.get(size=self.params.period))
            self.lines.upper[0] = highest_high
            self.lines.lower[0] = lowest_low

In [36]:
class EMAStrategy(bt.Strategy):
    params = (('fast_ema', 12), ('slow_ema', 26), ('signal_ema', 9), ('donchian_period', 20),)

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.fast_ema = bt.indicators.ExponentialMovingAverage(self.datas[0].close, period=self.params.fast_ema)
        self.slow_ema = bt.indicators.ExponentialMovingAverage(self.datas[0].close, period=self.params.slow_ema)
        self.signal_ema = bt.indicators.ExponentialMovingAverage(self.datas[0].close, period=self.params.signal_ema)
        self.donchian = DonchianChannel(self.datas[0])

    def next(self):
        if self.fast_ema > self.slow_ema and self.dataclose[0] > self.donchian.lines.upper:
            self.buy()
        elif self.fast_ema < self.slow_ema and self.dataclose[0] < self.donchian.lines.lower:
            self.sell()

In [37]:
cerebro = bt.Cerebro()
#modpath = os.path.dirname(os.path.abspath(sys.argv[0]))    
datapath = '../data/TSLA.csv'
data = bt.feeds.YahooFinanceCSVData(dataname=datapath, fromdate=datetime.datetime(2022, 10, 31), todate=datetime.datetime(2023, 4, 30),)  
cerebro.adddata(data)
cerebro.addstrategy(EMAStrategy)

cerebro.broker.set_cash(100000)
# Set the commission for buying and selling
cerebro.broker.setcommission(commission=0.001)

# Print the starting cash amount
print(f'Starting Portfolio Value: {cerebro.broker.getvalue()}')

cerebro.run()

print(f'Ending Portfolio Value: {cerebro.broker.getvalue()}')

Starting Portfolio Value: 100000
Ending Portfolio Value: 100000.0
